# NarrativeKoGPT2 학습

## 1.Google Drive 연동
- 모델 파일과 학습 데이터가 저장 되어있는 구글 드라이브의 디렉토리와 Colab을 연동.  
- 좌측상단 메뉴에서 런타임-> 런타임 유형 변경 -> 하드웨어 가속기 -> GPU 선택 후 저장

### 1.1 GPU 연동 확인

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



### 1.2 Google Drive 연동
아래 코드를 실행후 나오는 URL을 클릭하여 나오는 인증 코드 입력

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!ls

drive  sample_data


**Colab 디렉토리 아래 NarrativeKoGPT2 경로 확인**

In [4]:
!ls drive/'My Drive'/'Colab Notebooks'/

BERT_X	KorQuAD-beginner  NarrativeKoGPT2  NarrativeKoGPT2.ipynb


**필요 패키지들 설치**

In [4]:
!pip install -r drive/'My Drive'/'Colab Notebooks'/NarrativeKoGPT2/requirements.txt

In [2]:
import os

import sys
sys.path.append('drive/My Drive/Colab Notebooks/')
print(os.getcwd())


/content


## 2.KoGPT2 Transfer Learning

### 2.1.Import Package

In [3]:
import random
import torch
from torch.utils.data import DataLoader # 데이터로더
from gluonnlp.data import SentencepieceTokenizer 
from NarrativeKoGPT2.kogpt2.utils import get_tokenizer
from NarrativeKoGPT2.kogpt2.utils import download, tokenizer
from NarrativeKoGPT2.model.torch_gpt2 import GPT2Config, GPT2LMHeadModel
from NarrativeKoGPT2.util.data import NovelDataset
import gluonnlp

### 2.2. koGPT-2 Config

In [0]:
ctx= 'cpu'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cachedir='~/kogpt2/' # KoGPT-2 모델 다운로드 경로
epoch =200  # 학습 epoch

pytorch_kogpt2 = {
    'url':
    'https://kobert.blob.core.windows.net/models/kogpt2/pytorch/pytorch_kogpt2_676e9bcfa7.params',
    'fname': 'pytorch_kogpt2_676e9bcfa7.params',
    'chksum': '676e9bcfa7'
}
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000
}

### 2.3 Model and Vocab Download

In [5]:
# download model
model_info = pytorch_kogpt2
model_path = download(model_info['url'],
                       model_info['fname'],
                       model_info['chksum'],
                       cachedir=cachedir)
# download vocab
vocab_info = tokenizer
vocab_path = download(vocab_info['url'],
                       vocab_info['fname'],
                       vocab_info['chksum'],
                       cachedir=cachedir)

using cached model
using cached model


### 2.4.KoGPT-2 Model Vocab

In [0]:
# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
# model_path로부터 다운로드 받은 내용을 load_state_dict으로 업로드
kogpt2model.load_state_dict(torch.load(model_path))

device = torch.device(ctx)
kogpt2model.to(device)

# kogpt2model.eval()
# 추가로 학습하기 위해 .train() 사용
kogpt2model.train()
vocab_b_obj = gluonnlp.vocab.BERTVocab.from_sentencepiece(vocab_path,
                                                     mask_token=None,
                                                     sep_token=None,
                                                     cls_token=None,
                                                     unknown_token='<unk>',
                                                     padding_token='<pad>',
                                                     bos_token='<s>',
                                                     eos_token='</s>')


### 2.5. Get Batch Data using DataLoader

In [7]:
tok_path = get_tokenizer()
model, vocab = kogpt2model, vocab_b_obj
sentencepieceTokenizer = SentencepieceTokenizer(tok_path)

#os.chdir("../")
data_file_path = 'drive/My Drive/Colab Notebooks/NarrativeKoGPT2/data/backmyo_novel_1/untokenized_bm_data.txt'
batch_size = 4
novel_dataset = NovelDataset(data_file_path, vocab,sentencepieceTokenizer)
novel_data_loader = DataLoader(novel_dataset, batch_size=batch_size, shuffle=True)

using cached model
(905,)


### 2.6. Learning rate, Loss function, Adam Optimizer

In [0]:
learning_rate = 1e-5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


### 2.7. KoGPT-2 Transfer Laerning

In [0]:
print('KoGPT-2 Transfer Learning Start')
epoch=200
for epoch in range(epoch):
  count = 0
  for data in novel_data_loader:
    print('train no.{}'.format(count+1))
    optimizer.zero_grad()
    # train_data = torch.tensor([data])
    data = torch.stack(data) # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.

    data= data.transpose(1,0)
    # data= data.to(ctx)
    # print(data.shape)

    outputs = model(data, labels=data)
    loss, logits = outputs[:2]
    loss.backward()
    optimizer.step()
    if count %10 ==0:
      print('epoch no.{} train no.{}  loss = {}' . format(epoch, count+1, loss))
    count += 1

KoGPT-2 Transfer Learning Start
train no.1
epoch no.0 train no.1  loss = 4.408602237701416
train no.2
train no.3
train no.4
train no.5
train no.6
train no.7
train no.8
train no.9
train no.10
train no.11
epoch no.0 train no.11  loss = 4.399487018585205
train no.12
train no.13
train no.14
train no.15
train no.16
train no.17
train no.18
train no.19
train no.20
train no.21
epoch no.0 train no.21  loss = 4.43223762512207
train no.22
train no.23
train no.24
train no.25
train no.26
train no.27
train no.28
train no.29
train no.30
train no.31
epoch no.0 train no.31  loss = 4.016036033630371
train no.32
train no.33
train no.34
train no.35
train no.36
train no.37
train no.38
train no.39
train no.40
train no.41
epoch no.0 train no.41  loss = 4.198795795440674
train no.42
train no.43
train no.44
train no.45
train no.46
train no.47
train no.48
train no.49
train no.50
train no.51
epoch no.0 train no.51  loss = 4.073155403137207
train no.52
train no.53
train no.54
train no.55
train no.56
train no.57
t